--- Day 21: Allergen Assessment ---

You reach the train's last stop and the closest you can get to your vacation island without getting wet. There aren't even any boats here, but nothing can stop you now: you build a raft. You just need a few days' worth of food for your journey.

You don't speak the local language, so you can't read any ingredients lists. However, sometimes, allergens are listed in a language you do understand. You should be able to use this information to determine which ingredient contains which allergen and work out which foods are safe to take with you on your trip.

You start by compiling a list of foods (your puzzle input), one food per line. Each line includes that food's ingredients list followed by some or all of the allergens the food contains.

Each allergen is found in exactly one ingredient. Each ingredient contains zero or one allergen. Allergens aren't always marked; when they're listed (as in (contains nuts, shellfish) after an ingredients list), the ingredient that contains each listed allergen will be somewhere in the corresponding ingredients list. However, even if an allergen isn't listed, the ingredient that contains that allergen could still be present: maybe they forgot to label it, or maybe it was labeled in a language you don't know.

For example, consider the following list of foods:

```
mxmxvkd kfcds sqjhc nhms (contains dairy, fish)
trh fvjkl sbzzf mxmxvkd (contains dairy)
sqjhc fvjkl (contains soy)
sqjhc mxmxvkd sbzzf (contains fish)
```

The first food in the list has four ingredients (written in a language you don't understand): mxmxvkd, kfcds, sqjhc, and nhms. While the food might contain other allergens, a few allergens the food definitely contains are listed afterward: dairy and fish.

The first step is to determine which ingredients can't possibly contain any of the allergens in any food in your list. In the above example, none of the ingredients kfcds, nhms, sbzzf, or trh can contain an allergen. Counting the number of times any of these ingredients appear in any ingredients list produces 5: they all appear once each except sbzzf, which appears twice.

Determine which ingredients cannot possibly contain any of the allergens in your list. How many times do any of those ingredients appear?


In [1]:
#!fsharp
let testInput = """mxmxvkd kfcds sqjhc nhms (contains dairy, fish)
trh fvjkl sbzzf mxmxvkd (contains dairy)
sqjhc fvjkl (contains soy)
sqjhc mxmxvkd sbzzf (contains fish)"""

let taskInput = System.IO.File.ReadAllText("task.txt")

In [1]:
#!fsharp
type Food = (string array) * (string array)

let parse (input:string) =
    input.Replace("\r","").Split("\n")
    |> Seq.map
        (fun row ->
            let index = row.IndexOf " (contains "
            let ingrediences = row.[..(index-1)].Split(" ")
            let alergens = row.[(index+11) .. ^1].Split(", ")
            (ingrediences, alergens)
        )

testInput |> parse

index,Item1,Item2
0,"[ mxmxvkd, kfcds, sqjhc, nhms ]","[ dairy, fish ]"
1,"[ trh, fvjkl, sbzzf, mxmxvkd ]",[ dairy ]
2,"[ sqjhc, fvjkl ]",[ soy ]
3,"[ sqjhc, mxmxvkd, sbzzf ]",[ fish ]


In [1]:
#!fsharp
let findPotentialIngrediences (foods: Food seq) =
    let mutable result: Map<string, string Set> = Map.empty
    foods
    |> Seq.iter
        ( fun (ingrediences, alergens) -> 
            alergens
            |> Seq.iter
                (fun alergen -> 
                    let possible = 
                        ingrediences
                        |> Set.ofSeq
                    if result.ContainsKey alergen then
                        let newSet = Set.intersect result.[alergen] possible
                        result <- result.Add( alergen, newSet)
                    else
                        result <- result.Add( alergen, possible)
                )            
        )
    result

testInput |> parse |> findPotentialIngrediences

key,value
dairy,[ mxmxvkd ]
fish,"[ mxmxvkd, sqjhc ]"
soy,"[ fvjkl, sqjhc ]"


In [1]:
#!fsharp
taskInput |> parse |> findPotentialIngrediences

key,value
dairy,"[ dpkvsdk, drbq, rkcpxs, xmmpt ]"
eggs,"[ rkcpxs, xmmpt ]"
fish,"[ cxjqxbt, drbq, rkcpxs, xmmpt ]"
nuts,"[ drbq, xmmpt ]"
peanuts,[ zmzq ]
sesame,"[ dpkvsdk, drbq, mnrjrf ]"
shellfish,"[ kjgl, mnrjrf, zmzq ]"
wheat,"[ rkcpxs, zmzq ]"


In [1]:
#!fsharp
let convertToPotentialAlergens (map: Map<string, string Set>) =
    let mutable result: Map<string, string Set> = Map.empty
    map
    |> Map.iter (
        fun key set ->
            set
            |> Set.iter
                ( fun ingredience ->
                    if result.ContainsKey ingredience then
                        let newSet = result.[ingredience].Add(key)
                        result <- result.Add( ingredience, newSet )
                    else
                        result <- result.Add( ingredience, (Set.ofList [key]) )
                )        
    )
    result

testInput |> parse |> findPotentialIngrediences |> convertToPotentialAlergens

key,value
fvjkl,[ soy ]
mxmxvkd,"[ dairy, fish ]"
sqjhc,"[ fish, soy ]"


In [1]:
#!fsharp
let findSafeFoodCount (food: Food seq) =
    let alergenMap =
        food
        |> findPotentialIngrediences
        |> convertToPotentialAlergens
    
    food
    |> Seq.sumBy 
        (fun (ingrediences, alergens) ->
            ingrediences
            |> Seq.filter (fun ingredience -> not (alergenMap.ContainsKey ingredience))
            |> Seq.length
        )

testInput |> parse |> findSafeFoodCount

5

In [1]:
#!fsharp
taskInput |> parse |> findSafeFoodCount

1977

--- Part Two ---

Now that you've isolated the inert ingredients, you should have enough information to figure out which ingredient contains which allergen.

In the above example:

*    mxmxvkd contains dairy.
*    sqjhc contains fish.
*    fvjkl contains soy.

Arrange the ingredients alphabetically by their allergen and separate them by commas to produce your canonical dangerous ingredient list. (There should not be any spaces in your canonical dangerous ingredient list.) In the above example, this would be mxmxvkd,sqjhc,fvjkl.

Time to stock your raft with supplies. What is your canonical dangerous ingredient list?


In [1]:
#!fsharp
let finalizeAlergens (map: Map<string, string Set>) =
    let mutable remaining = Map.toList map
    let mutable result: Map<string, string> = Map.empty
    while not (remaining.IsEmpty) do
        let resolved = 
            remaining
            |> List.filter
                (fun (alergen, ingrediences) ->
                    ingrediences.Count = 1
                )
        
        remaining <- remaining
        |> List.map 
            (fun (alergen, ingrediences) ->
                let mutable newIngrediences = ingrediences

                resolved 
                |> List.iter
                    (fun (_, ing) ->
                        newIngrediences <- newIngrediences.Remove (ing.MinimumElement)
                    )

                (alergen, newIngrediences)
            )
        |> List.filter
            (fun (alergen, ingrediences) -> not ingrediences.IsEmpty)

        resolved
        |> List.iter 
            ( fun (alergen,ingrediences) ->
                result <- result.Add( ingrediences.MinimumElement, alergen)
            )

    result

testInput |> parse |> findPotentialIngrediences |> finalizeAlergens

key,value
fvjkl,soy
mxmxvkd,dairy
sqjhc,fish


In [1]:
#!fsharp
taskInput |> parse |> findPotentialIngrediences |> finalizeAlergens

key,value
cxjqxbt,fish
dpkvsdk,dairy
drbq,nuts
kjgl,shellfish
mnrjrf,sesame
rkcpxs,wheat
xmmpt,eggs
zmzq,peanuts


In [1]:
#!fsharp
let getDangerousList (alergens: Map<string,string>) =
    alergens
    |> Map.toList
    |> List.sortBy snd
    |> List.map fst
    |> String.concat ","

testInput |> parse |> findPotentialIngrediences |> finalizeAlergens |> getDangerousList

mxmxvkd,sqjhc,fvjkl

In [1]:
#!fsharp
taskInput |> parse |> findPotentialIngrediences |> finalizeAlergens |> getDangerousList

dpkvsdk,xmmpt,cxjqxbt,drbq,zmzq,mnrjrf,kjgl,rkcpxs